In [78]:
import pandas as pd
import numpy as np

In [79]:
df = pd.read_csv('OriginalData/skills.csv')

In [80]:
df

,user_id,skill
0,1,Mühendislik
1,1,Eğitim
2,2,Android
3,2,Java
4,2,3D Studio Max
...,...,...
1398438,66273,CI/CD
1398439,66273,Terraform
1398440,66273,MongoDB
1398441,66273,Go (Programming Language)


## Data prep
- Column for number of skills
- Categorize skills into broad categories (IT, Soft skills, etc)
 

Number of skills- will add later once I have a single entry for each user

In [81]:
n_skills = df["user_id"].value_counts().reset_index().rename(columns={'index': 'user_id', 'user_id': 'N_skills'})

### Categorizing Skills

In [82]:
skills = df.skill.unique().tolist()
skills.sort()
print(len(skills))
skills

53025


['\tComputer and Network Security',
 '\tProject and Change Management',
 '\tSystems Analysis, Model. and Design',
 '   C, C++',
 '   English',
 '   Tasarım Özellikleri',
 '  Back-End Web Development',
 '  CSS',
 '  ElasticSearch',
 '  Entity Framework',
 '  Fibre Channel ',
 '  Git',
 '  Google Webmaster Tools ',
 '  Incident Management',
 '  JIRA',
 '  Kibana',
 '  Linux',
 '  Logstash',
 '  Microsoft SQL Server',
 '  MySQL',
 '  Object Oriented Design Pattern',
 '  Python',
 '  React Native',
 '  Rest',
 '  SEM ',
 '  SEO ',
 '  SQLite',
 '  Semiconductors',
 '  Software Development ',
 '  Software Engineering ',
 '  Terraform',
 '  UML ',
 '  Visual Studio',
 '  XML',
 '  Yazılım Geliştirme',
 ' . Net Core',
 ' .NET',
 ' .NET  ',
 ' .NET Core',
 ' .NET Framework',
 ' .Net',
 ' .Net Core',
 ' 3D Modelling',
 ' ADO.NET',
 ' ADO.Net ',
 ' AJAX',
 ' API Testing',
 ' ASP.NET',
 ' ASP.NET Core',
 ' ASP.NET MVC',
 ' ASP.NET Web API',
 ' ASP.NET WebAPI',
 ' Active Information Gathering',
 '

In [83]:
skills_groups = {
    "IT":["Web", "Programming", "C", "C++", "Python", "Java", ".NET", "command line", "yapay zeka", "Project", "Testing", "Git", "Oyun", "Development", 
    "LaTeX", "Math", "Technology", "Data", "Database", "Mühendis", "Sorun", "Object Oriented", "Code", "Linux", "Windows", "Mac", "Android", "iOS", "SQL",
     "Database", "Network", "Security", "Cloud", "DevOps", "Data", "Machine Learning", "internet"],
    "Business":["Finance", "Marketing", "Business", "İş", "Pazarlama", ],
    "Design": ["Design", "Photoshop", "Illustrator", "InDesign", "After Effects", "Premiere", "3D", "Animation", "Video", "Audio", "Motion Graphics"]
}

In [84]:
from fuzzywuzzy import fuzz
from collections import defaultdict

def group_strings(strings, reference_strings, similarity_threshold):
    groups = {key: [] for key in reference_strings.keys()}
    groups["Other"] = []
    key_list = [key for key in reference_strings.keys()]
    for i, string1 in enumerate(strings):
        max_similarity = 0
        similarities = []
        for key, values in reference_strings.items():
            similarities.append(max([fuzz.token_sort_ratio(string1, val) for val in values]))
        idx = np.argmax(similarities)
        if similarities[idx] >= similarity_threshold:
            groups[key_list[idx]].append((string1,similarities[idx]))
        else:
            groups["Other"].append((string1,similarities[idx]))
    for key in groups.keys():
        groups[key] = [string for string, sim in sorted(groups[key], key=lambda x: -x[1])]
    return groups

In [85]:
groups = group_strings(skills, skills_groups, 60)

In [86]:
for key,value in groups.items():
    df.loc[df.skill.isin(groups[key]), "skill"]  = key

Create columns to indicate number of skills

In [87]:
df = df.groupby('user_id')['skill'].value_counts().unstack().fillna(0)
df = df.astype(int)
df.reset_index(inplace=True,drop=False)
df["N_Skills"] = df.Business + df.Design + df.IT + df.Other

Save

In [88]:
df.to_csv("PreparedData/skills.csv", index=False)